In [1]:
import os
import json


In [2]:
# Open and read the config file
with open('config.json', 'r') as config_file:
    config_data = json.load(config_file)

# Retrieve the API key from the config data
api_key = config_data['api_key']
os.environ['OPENAI_API_KEY'] = api_key

## Load the index

In [3]:
from llama_index import StorageContext, load_index_from_storage

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="./storage")
# load index
index = load_index_from_storage(storage_context)

## Query index

In [4]:
# create query engine
# it will use lazy embedding
queryEngine = index.as_query_engine()

In [5]:
queryEngine.query("how do I plan a tunneling project in LiquidEarth? Try finding the Control Fields for each steps of the workflow and append them as a list in order of the operations to your response. only include controls that you are confident exist and are relevant to the question. Only answer this question if you can find the answer in the documentation. If no documentation is found, ask the user to rephrase the question.")

Response(response='\n1. Select the “Tunneling” option from the “Projects” tab.\n2. Select the “Create” button to start a new project.\n3. Select the “Import” button to upload the 3D geological data.\n4. Select the “Analyze” button to analyze the data.\n5. Select the “Design” button to design the tunneling project.\n6. Select the “Simulate” button to simulate the project.\n7. Select the “Export” button to export the project.', source_nodes=[NodeWithScore(node=TextNode(id_='f15bb996-8bcd-4113-b0fb-e3e87b27debd', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='2aebf52a-532c-4862-983f-830390801537', node_type=None, metadata={}, hash='e23fdfe472529d28782acc176c21be8a7053496336abdf00326a04e7c106a788')}, hash='97bbeea97343bad39518559157513c9a3250c3b321b872880aba521b8b7e14af', text='LiquidEarth Documentation\n\nAll things related to the product - processes, best practices, setup

## Query with a prompt template

In [7]:
from llama_index import Prompt
# define custom Prompt
TEMPLATE_STR = (
    "We have provided Documentation on the software and further information below. The Documentation includes Control properties, with the name of the UI element for a certain functionality within the LiquidEarth App .\n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Answer the question for a human to understand. Additionally, ONLY where controls are available and, return the control properties in the order of operations in the following form at the end of your response: [[control1], [control2], ...]. Never include any controls that are empty or not specifically included as Control properties in the provided documentation. Given this information, please answer the question: {query_str}\n"
)
QA_TEMPLATE = Prompt(TEMPLATE_STR)

# Configure query engine
query_engine = index.as_query_engine(text_qa_template=QA_TEMPLATE)

# Execute query
response = query_engine.query("how can I create a project?")
print(response)


To create a project, open up the server explorer on the left and click the Plus button to create a new project. It will appear at the bottom of your server explorer list. To adapt basic information and project metadata (such as project name and owner), use Edit Data in the inspector. Once you have set all required fields correctly, Save them to accept the information. Then, use Load Project in the inspector to load it into your local workspace (3D Workspace). It will appear in your 3D workspace and listed in the Local Explorer. [[Plus], [Edit Data], [Save], [Load Project]].


In [28]:
response = query_engine.query("how to install liquidEarth on my quest?")

In [29]:
print(response)


To install LiquidEarth on your Quest device, you will need to have a valid Meta/Oculus account. First, submit the email address associated with your Oculus/Meta account via the respective text field. Your account will then be added to the App Lab Beta channel for LiquidEarth. This process can take up to one working day and you will receive a confirmation e-mail. In the confirmation e-mail, activate the verification link and log in with your Meta account. Ensure your Quest device is connected to the internet and that you are signed into it with the correct account. Now LiquidEarth will appear in the Quest Store if you are logged in with your account and search for “LiquidEarth” using the search box using either a web browser, the Meta Quest app on your phone, or the Store on your Quest Device. On the Store on the Quest device, click the LiquidEarth app tile to install it. Once installed, the LiquidEarth app will appear in your App Library. Alternatively you can use the Meta Quest app o